In [2]:
%env CUDA_VISIBLE_DEVICES=0

env: CUDA_VISIBLE_DEVICES=0


In [3]:
from synthesizer.inference import Synthesizer
from synthesizer_like_translator.inference import Synthesizer as Translator
from synthesizer.kaldi_interface import KaldiInterface
from encoder import inference as encoder
from encoder import inference_accent as encoder_accent
from vocoder import inference as vocoder
import numpy as np
from pathlib import Path
from utils.argutils import print_args
import random
import IPython.display as ipd
from synthesizer.hparams import hparams
import librosa
import matplotlib.pyplot as plt
from matplotlib import offsetbox
from sklearn import (manifold, datasets, decomposition, ensemble, discriminant_analysis, random_projection)


encoder_speaker_weights = Path("/home/grads/q/quamer.waris/projects/Accentron/pretrained_model/pretrained/encoder/saved_models/pretrained.pt")
encoder_accent_weights = Path("/home/grads/q/quamer.waris/projects/Accentron/pretrained_model/pretrained/encoder/saved_models/encoder_accent.pt")
vocoder_weights = Path("/home/grads/q/quamer.waris/projects/Accentron/pretrained_model/pretrained/vocoder/saved_models/pretrained/pretrained.pt")
#syn_dir = Path('/mnt/data1/waris/model_outputs/accentron/parallel_vctk_mic2/logs-Accetron_train_parallel_vctk/taco_pretrained')
syn_dir = Path("/mnt/data1/waris/model_outputs/accentron/parallel/logs-Accetron_train_parallel/taco_pretrained")
#syn_dir_trans = Path("/mnt/data1/waris/model_outputs/translator/sythesizer_like/logs-translator_train/taco_pretrained")
syn_dir_trans = Path("/mnt/data1/waris/model_outputs/translator/sythesizer_like_train_set/logs-translator_train/taco_pretrained")

encoder.load_model(encoder_speaker_weights)
encoder_accent.load_model(encoder_accent_weights)
synthesizer = Synthesizer(syn_dir)
translator = Translator(syn_dir_trans)
vocoder.load_model(vocoder_weights)
#hparams = hparams.parse()

Instructions for updating:
non-resource variables are not supported in the long term


/home/grads/q/quamer.waris/anaconda3/envs/autospeech/lib/python3.8/site-packages/librosa/util/decorators.py:9: NumbaDeprecationWarning: An import was requested from a module that has moved location.
Import requested from: 'numba.decorators', please update to use 'numba.core.decorators' or pin to Numba version 0.48.0. This alias will not be present in Numba version 0.50.0.
  from numba.decorators import jit as optional_jit
/home/grads/q/quamer.waris/anaconda3/envs/autospeech/lib/python3.8/site-packages/librosa/util/decorators.py:9: NumbaDeprecationWarning: An import was requested from a module that has moved location.
Import of 'jit' requested from: 'numba.decorators', please update to use 'numba.core.decorators' or pin to Numba version 0.48.0. This alias will not be present in Numba version 0.50.0.
  from numba.decorators import jit as optional_jit


Loaded encoder "pretrained.pt" trained to step 1564501
Loaded encoder "encoder_accent.pt" trained to step 90001
Found synthesizer "Accetron_train_parallel" trained to step 204001
Found synthesizer "translator_train" trained to step 294001
Building Wave-RNN
Trainable Parameters: 4.481M
Loading model weights at /home/grads/q/quamer.waris/projects/Accentron/pretrained_model/pretrained/vocoder/saved_models/pretrained/pretrained.pt


In [4]:
def synthesize(bnf, embed):
    spec = synthesizer.synthesize_spectrograms([bnf], [embed])[0]
    # print(spec.shape)
    generated_wav = vocoder.infer_waveform(spec)
    generated_wav = np.pad(generated_wav, (0, synthesizer.sample_rate), mode="constant")
    return generated_wav

def translate_ppg(bnf, embed):
    spec = translator.synthesize_spectrograms([bnf], [embed])[0]
    return spec

def generate_accent_embed(src_utterance_path):
    wav, _ = librosa.load(src_utterance_path, hparams.sample_rate)
    wav = encoder.preprocess_wav(wav)
    embed_accent = encoder_accent.embed_utterance(wav)

    return embed_accent

def generate_speaker_embed(tgt_utterance_path):
    wav, _ = librosa.load(tgt_utterance_path, hparams.sample_rate)
    wav = encoder.preprocess_wav(wav)
    embed_speaker = encoder.embed_utterance(wav)

    return embed_speaker

In [10]:
english_utterances = ['/mnt/data1/waris/datasets/vctk/wav48_silence_trimmed/p226/wav/',
                        '/mnt/data1/waris/datasets/vctk/wav48_silence_trimmed/p227/wav/',
                        '/mnt/data1/waris/datasets/vctk/wav48_silence_trimmed/p232/wav/',
                        '/mnt/data1/waris/datasets/vctk/wav48_silence_trimmed/p256/wav/',
                        '/mnt/data1/waris/datasets/vctk/wav48_silence_trimmed/p260/wav/',
                        '/mnt/data1/waris/datasets/vctk/wav48_silence_trimmed/p263/wav/']

japanese_utterances = ['/mnt/data1/waris/datasets/jvs_ver1/jvs001/parallel100/wav24kHz16bit/',
                        '/mnt/data1/waris/datasets/jvs_ver1/jvs003/parallel100/wav24kHz16bit/',
                        '/mnt/data1/waris/datasets/jvs_ver1/jvs005/parallel100/wav24kHz16bit/',
                        '/mnt/data1/waris/datasets/jvs_ver1/jvs006/parallel100/wav24kHz16bit/',
                        '/mnt/data1/waris/datasets/jvs_ver1/jvs013/parallel100/wav24kHz16bit/',
                        '/mnt/data1/waris/datasets/jvs_ver1/jvs028/parallel100/wav24kHz16bit/']

mandarian_utterances = ['/mnt/data1/waris/datasets/UEDIN_mandarin_bi_data_2010/downsampled_22kHz/Mandarin_talkers/Male/MM1/Man/',
                        '/mnt/data1/waris/datasets/UEDIN_mandarin_bi_data_2010/downsampled_22kHz/Mandarin_talkers/Male/MM2/Man/',
                        '/mnt/data1/waris/datasets/UEDIN_mandarin_bi_data_2010/downsampled_22kHz/Mandarin_talkers/Male/MM3/Man/',
                        '/mnt/data1/waris/datasets/UEDIN_mandarin_bi_data_2010/downsampled_22kHz/Mandarin_talkers/Male/MM4/Man/',
                        '/mnt/data1/waris/datasets/UEDIN_mandarin_bi_data_2010/downsampled_22kHz/Mandarin_talkers/Male/MM5/Man/',
                        '/mnt/data1/waris/datasets/UEDIN_mandarin_bi_data_2010/downsampled_22kHz/Mandarin_talkers/Male/MM6/Man/']

mandarian_eng_utterances = ['/mnt/data1/waris/datasets/mandarin_bi_speakers/English/MM1/wav/',
                        '/mnt/data1/waris/datasets/mandarin_bi_speakers/English/MM2/wav/',
                        '/mnt/data1/waris/datasets/mandarin_bi_speakers/English/MM3/wav/',
                        '/mnt/data1/waris/datasets/mandarin_bi_speakers/English/MM4/wav/',
                        '/mnt/data1/waris/datasets/mandarin_bi_speakers/English/MM5/wav/',
                        '/mnt/data1/waris/datasets/mandarin_bi_speakers/English/MM6/wav/']

In [6]:
speaker_embeddings = []
speaker_labels = []

for utterance in english_utterances:
    path_eng, dir_eng, files_eng = next(os.walk(utterance))
    count = 0
    for i, file in enumerate(files_eng):
        if count>=100:
            break

        if "_mic2.wav" in file: 
            embed_speaker = generate_speaker_embed(path_eng + file)
            speaker_embeddings.append(embed_speaker)
            speaker_labels.append("ENG")
            count=count+1


for utterance in japanese_utterances:
    path_jap, dir_jap, files_jap = next(os.walk(utterance))

    for i, file in enumerate(files_jap):
        embed_speaker = generate_speaker_embed(path_jap + file)
        speaker_embeddings.append(embed_speaker)
        speaker_labels.append("JAP")

for utterance in mandarian_utterances:
    path_man, dir_man, files_man = next(os.walk(utterance))
    count = 0
    for i, file in enumerate(files_man):
        if count>=100:
            break
        if "_1.wav" in file: 
            embed_speaker = generate_speaker_embed(path_man + file)
            speaker_embeddings.append(embed_speaker)
            speaker_labels.append("MAN")
            count=count+1


In [7]:
speaker_embeddings_copy = speaker_embeddings
speaker_labels_copy = speaker_labels

1800

In [11]:
for utterance in mandarian_eng_utterances:
    path_man, dir_man, files_man = next(os.walk(utterance))
    count = 0
    for i, file in enumerate(files_man):
        if count>=10:
            break
        if "_1.wav" in file: 
            embed_speaker = generate_speaker_embed(path_man + file)
            speaker_embeddings.append(embed_speaker)
            speaker_labels.append("MAN_ENG")
            count=count+1

In [13]:
### Computing PCA
print("Computing PCA projection")
X_pca = decomposition.TruncatedSVD(n_components=2).fit_transform(speaker_embeddings)
## Computing t-SNE
print("Computing t-SNE embedding")
tsne = manifold.TSNE(n_components=2, init='pca', random_state=0)
speaker_tsne = tsne.fit_transform(speaker_embeddings)

Computing PCA projection
Computing t-SNE embedding


In [14]:
colors = ['tab:blue', 'tab:green', 'tab:red', 'tab:gray']
shapes = ["o", "^", "*", "s"]
labels = ["ENG", "JAP", "MAN", "MAN_ENG"]

In [15]:
import matplotlib.cm as cm

plt.figure(figsize=(12,8))
for speaker, c, m in zip(labels, colors, shapes):
    X_speaker_embedding = X_pca[np.where(speaker==np.array(speaker_labels))]
    print(X_speaker_embedding.shape, speaker)
    plt.scatter(X_speaker_embedding[:,0], X_speaker_embedding[:,1], label=speaker, marker=m, color=c)
    # plt.scatter(speaker_embeddings[:, 0], speaker_embeddings[:, 1], c=speakers_all)
plt.legend()
plt.tight_layout()
plt.savefig("SpeakerEmbeddings_Jap_Eng_BiMan_PCA.png", format='png')

(600, 2) ENG
(600, 2) JAP
(600, 2) MAN
(60, 2) MAN_ENG


In [16]:
import matplotlib.cm as cm

plt.figure(figsize=(12,8))
for speaker, c, m in zip(labels, colors, shapes):
    X_speaker_embedding = speaker_tsne[np.where(speaker==np.array(speaker_labels))]
    print(X_speaker_embedding.shape, speaker)
    plt.scatter(X_speaker_embedding[:,0], X_speaker_embedding[:,1], label=speaker, marker=m, color=c)
    # plt.scatter(speaker_embeddings[:, 0], speaker_embeddings[:, 1], c=speakers_all)
plt.legend()
plt.tight_layout()
plt.savefig("SpeakerEmbeddings_Jap_Eng_BiMan_TSNE.png", format='png')

(600, 2) ENG
(600, 2) JAP
(600, 2) MAN
(60, 2) MAN_ENG


In [19]:
import os
from scipy.io import wavfile
import numpy as np


# utterance_ids = ['VOICEACTRESS100_001', 'VOICEACTRESS100_002', 'VOICEACTRESS100_003', 'VOICEACTRESS100_004', 'VOICEACTRESS100_005', 'VOICEACTRESS100_006']
# speakers = ['jvs001', 'jvs003']

speakers = ['MM1', 'MM2','MM3', 'MM4', 'MM5', 'MM6']

# for utterance in mandarian_eng_utterances:
#     path_man, dir_man, files_man = next(os.walk(utterance))
#     count = 0
#     for i, file in enumerate(files_man):
#         if count>=100:
#             break
#         if "_1.wav" in file: 
#             embed_speaker = generate_speaker_embed(path_man + file)
#             speaker_embeddings.append(embed_speaker)
#             speaker_labels.append("MAN")
#             count=count+1

for speaker in speakers:
    utterance_fpath = '/mnt/data1/waris/datasets/mandarin_bi_speakers/English/'+speaker+'/wav'
    _, _, utterance_ids = next(os.walk(utterance_fpath))
    count = 0
    for utterance_id in utterance_ids:
        if count>=10:
            break

        if "_1.wav" in utterance_id:
            kaldi_dir = '/mnt/data1/waris/datasets/mandarin_bi_speakers/English/'+speaker+'/kaldi'
            ki = KaldiInterface(wav_scp=str(os.path.join(kaldi_dir, 'wav.scp')),
                                bnf_scp=str(os.path.join(kaldi_dir, 'bnf/feats.scp')))
            bnf = ki.get_feature('_'.join([speaker, utterance_id[:-4]]), 'bnf')

            acc_utterance_path = '/mnt/data1/waris/datasets/data/arctic_dataset/BDL/wav/arctic_b0534.wav'
            embed_accent = generate_accent_embed(acc_utterance_path)
            bnf_native = translate_ppg(bnf, embed_accent)

            tgt_utterance_path = '/mnt/data1/waris/datasets/mandarin_bi_speakers/English/'+speaker+'/wav/'+str(utterance_id)
            embed_speaker = generate_speaker_embed(tgt_utterance_path)

            synthesis_wav = synthesize(bnf_native, embed_speaker)

            output_dir = '/home/grads/q/quamer.waris/projects/ac-vc/synthesis_output/crosslingual_analysis/'+str(speaker)
            if not os.path.isdir(output_dir):
                os.makedirs(output_dir)
            filename = str(utterance_id)
            output_file = os.path.join(output_dir, filename)
            wavfile.write(output_file, hparams.sample_rate, synthesis_wav)

            embed_speaker = generate_speaker_embed(output_file)
            speaker_embeddings.append(embed_speaker)
            speaker_labels.append("ENG_MAN")
            count = count + 1

{| ████████████████ 47500/48000 | Batch Size: 5 | Gen Rate: 5.1kHz | } | }

In [20]:
### Computing PCA
print("Computing PCA projection")
X_pca = decomposition.TruncatedSVD(n_components=2).fit_transform(speaker_embeddings)
## Computing t-SNE
print("Computing t-SNE embedding")
tsne = manifold.TSNE(n_components=2, init='pca', random_state=0)
speaker_tsne = tsne.fit_transform(speaker_embeddings)

Computing PCA projection
Computing t-SNE embedding


In [21]:
colors = ['tab:blue', 'tab:green', 'tab:red', 'tab:gray', 'tab:brown']
shapes = ["o", "^", "*", "s", "D"]
labels = ["ENG", "JAP", "MAN", "MAN_ENG", "ENG_MAN"]

In [22]:
import matplotlib.cm as cm

plt.figure(figsize=(12,8))
for speaker, c, m in zip(labels, colors, shapes):
    X_speaker_embedding = X_pca[np.where(speaker==np.array(speaker_labels))]
    print(X_speaker_embedding.shape, speaker)
    plt.scatter(X_speaker_embedding[:,0], X_speaker_embedding[:,1], label=speaker, marker=m, color=c)
    # plt.scatter(speaker_embeddings[:, 0], speaker_embeddings[:, 1], c=speakers_all)
plt.legend()
plt.tight_layout()
plt.savefig("SpeakerEmbeddings_Jap_Eng_BiMan_Corrected_PCA.png", format='png')

(600, 2) ENG
(600, 2) JAP
(600, 2) MAN
(60, 2) MAN_ENG
(60, 2) ENG_MAN


In [23]:
import matplotlib.cm as cm

plt.figure(figsize=(12,8))
for speaker, c, m in zip(labels, colors, shapes):
    X_speaker_embedding = speaker_tsne[np.where(speaker==np.array(speaker_labels))]
    print(X_speaker_embedding.shape, speaker)
    plt.scatter(X_speaker_embedding[:,0], X_speaker_embedding[:,1], label=speaker, marker=m, color=c)
    # plt.scatter(speaker_embeddings[:, 0], speaker_embeddings[:, 1], c=speakers_all)
plt.legend()
plt.tight_layout()
plt.savefig("SpeakerEmbeddings_Jap_Eng_BiMan_Corrected_TSNE.png", format='png')

(600, 2) ENG
(600, 2) JAP
(600, 2) MAN
(60, 2) MAN_ENG
(60, 2) ENG_MAN


In [3]:
import os
import shutil

_, dir_speakers, _ = next(os.walk('/mnt/data1/waris/model_results/vc_vctk/converted_utterances'))

root_path = '/mnt/data1/waris/model_results/vc_vctk_samples/converted_utterances/'
for speaker in dir_speakers:
    _, _, utterances = next(os.walk(root_path+speaker))

    target_dir = '/mnt/data1/waris/model_results/vc_vctk_samples/target_split/seen/'+speaker
    if not os.path.isdir(target_dir):
        os.makedirs(target_dir)

    for utterance in utterances:
        src_path = '/mnt/data1/waris/datasets/vctk/wav48_silence_trimmed/'+speaker+"/wav/"+speaker+"_"+utterance
        shutil.copy(src_path, target_dir)

    o_path, _ , orginal_utterances = next(os.walk('/mnt/data1/waris/datasets/vctk/wav48_silence_trimmed/'+speaker+'/wav/'))

    target_dir = '/mnt/data1/waris/model_results/vc_vctk_samples/target_split/unseen/'+speaker
    if not os.path.isdir(target_dir):
        os.makedirs(target_dir)
    count = 0
    for utterance in orginal_utterances:
        if count >= 50:
            break
        if "_mic2.wav" in utterance and utterance[-12:] not in utterances:
            shutil.copy(o_path+utterance, target_dir)
            count = count+1

print("Done!")

# _, _, src_utterances = next(os.walk("/mnt/data1/waris/datasets/vctk/wav48_silence_trimmed/p250/wav/"))

# _, _, used_utterances = next(os.walk("/mnt/data1/waris/model_results/vc_vctk_samples/source_utterances/"))

# count = 0
# for utterance in src_utterances:
#     if count >=50:
#         break
#     if "_mic2.wav" in utterance and utterance not in used_utterances:
#         shutil.copy('/mnt/data1/waris/datasets/vctk/wav48_silence_trimmed/p250/wav/'+utterance, '/mnt/data1/waris/model_results/vc_vctk/source_utterances')
#         count = count + 1
# print("Done!")


StopIteration: 

In [5]:
def intersection(lst1, lst2):
    return list(set(lst1) & set(lst2))

In [6]:
import os
import shutil


_, speaker_dirs , _ = next(os.walk('/mnt/data1/waris/model_results/vc_vctk/converted_utterances'))

for speaker in speaker_dirs:
    
    



['039_mic2.wav', '282_mic2.wav', '271_mic2.wav', '298_mic2.wav', '063_mic2.wav', '134_mic2.wav', '205_mic2.wav', '242_mic2.wav', '308_mic2.wav', '212_mic2.wav', '002_mic2.wav', '069_mic2.wav', '111_mic2.wav', '302_mic2.wav', '222_mic2.wav', '005_mic2.wav']


In [ ]:
/mnt/data1/waris/model_results/vctk_vc